In [1]:
import random 
import numpy as np 
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split
from utils.transformation import *

VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEAN = '../data/cleaning'

random.seed(VAR_SEED)
np.random.seed(VAR_SEED)

catalogo = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/catalogo.csv", sep=",", encoding="latin1")
mf_dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/mf_dataset.csv", sep=",", encoding="latin1")

In [2]:
catalogo = calculate_ratio_of_interactions(mf_dataset, catalogo)

# Parámetros para el cálculo del puntaje
features = {"hito": (1, 4), "skill": (0, 15), "knowledge": (1, 15), "interaction_ratio": (0.0, 1.0)}
weights = {"hito": 0.3, "skill": 0.25, "knowledge": 0.25, "interaction_ratio": 0.2}

# Calcular el puntaje en el DataFrame
catalogo = calculate_score_dataset(catalogo, features, weights, new_column='formula')

scale_complexity_norm = MinMaxScaler()
catalogo['complexity_norm'] = scale_complexity_norm.fit_transform(catalogo[['complexity']])

scale_complexity12_norm = MinMaxScaler()
catalogo['complexity12_norm'] = scale_complexity12_norm.fit_transform(catalogo[['complexity12']])

df_catalogo = catalogo[['oid', 'nombre', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'formula', 'complexity_norm', 'complexity12_norm']]


In [5]:
df_ratings = factorization_to_ratings(df_catalogo, mf_dataset, user_col='rut')
df_ratings.head(10)

,rut,oid,nombre,hito,skill,knowledge,complexity,complexity12,formula,complexity_norm,complexity12_norm
0,0,0973dae0e1b74ab8baa8d94339ee3ae6,CÃ¡lculo del dÃ­gito verificador del rut,1,1,7,23,279,0.281251,0.111111,0.022774
1,0,16f619db31204ded9418136c4587ddd8,Calculadora GeomÃ©trica,2,1,1,17,529,0.188468,0.080808,0.281573
2,0,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,1,0,3,3,259,0.177229,0.010101,0.002070
3,0,2437df93d3f44a87b00834072aeb1ab0,Nota Final,1,0,1,1,257,0.191384,0.000000,0.000000
4,0,29f15ef8dc32426f945f64e28c910a57,Suma de los divisores de un nÃºmero,2,1,3,19,531,0.224705,0.090909,0.283644
5,0,46850a246d48484b8f104f8aab5679b6,Descomponer un nÃºmero,1,0,3,3,259,0.192111,0.010101,0.002070
6,0,718578451f3f4eca87437cadfe98d688,Suma de los N primeros nÃºmeros naturales,1,0,2,2,258,0.210285,0.005051,0.001035
7,0,729d37da8f2d46f3af2d891df04949ef,Juego Adivina mi nÃºmero,1,0,3,3,259,0.137281,0.010101,0.002070
8,0,7f60644b0a1b484681ae5c8e36166c58,Signo del ZodÃ­aco,1,1,1,17,273,0.186379,0.080808,0.016563
9,0,80c61dae74fa4915bf272ab17dfa62ff,Conversor de Decimal a Binario,1,0,2,2,258,0.171904,0.005051,0.001035


In [6]:
# ESCALAS DE RANKING 
escala_01 = Reader(rating_scale=(0, 1))
escala_02 = Reader(rating_scale=(1, 255))
escala_03 = Reader(rating_scale=(257, 1279))

# CARGA DE DATOS
data_formula = Dataset.load_from_df(df_ratings[["rut", "oid", "formula"]], escala_01)
data_complexity = Dataset.load_from_df(df_ratings[["rut", "oid", "complexity"]], escala_02)
data_complexity12 = Dataset.load_from_df(df_ratings[["rut", "oid", "complexity12"]], escala_03)

# MODELOS
benchmark_formula = evaluate_algorithms(data_formula)
benchmark_complexity = evaluate_algorithms(data_complexity)
benchmark_complexity12 = evaluate_algorithms(data_complexity12)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


In [7]:
benchmark_formula

,test_rmse,fit_time,test_time,Benchmark Score
Algorithm,,,,
KNNBasic,0.000300,0.019709,0.216188,3329.592781
KNNBaseline,0.000888,0.023298,0.252699,1125.818595
BaselineOnly,0.004507,0.003017,0.004065,221.896348
SVDpp,0.014767,0.067740,0.026316,67.718609
KNNWithMeans,0.016288,0.024695,0.233804,61.395262
KNNWithZScore,0.022494,0.031430,0.238759,44.455442
SVD,0.043166,0.026950,0.006082,23.166295
SlopeOne,0.046607,0.003393,0.010733,21.455796
NMF,0.061214,0.043596,0.005139,16.336101


In [8]:
benchmark_complexity

,test_rmse,fit_time,test_time,Benchmark Score
Algorithm,,,,
SlopeOne,0.329358,0.003346,0.011068,3.036211
KNNBasic,0.397606,0.021156,0.235963,2.515055
KNNBaseline,0.510691,0.027614,0.262185,1.958131
SVDpp,1.280093,0.073083,0.027265,0.781193
SVD,2.134187,0.026666,0.006209,0.468562
BaselineOnly,2.589034,0.002976,0.004451,0.386244
CoClustering,4.234634,0.042104,0.006440,0.236148
KNNWithMeans,4.353310,0.023610,0.234435,0.229710
KNNWithZScore,6.197959,0.030390,0.239177,0.161343


In [9]:
benchmark_complexity12

,test_rmse,fit_time,test_time,Benchmark Score
Algorithm,,,,
KNNBasic,2.730082,0.019747,0.208681,0.366289
SlopeOne,3.182421,0.003762,0.011034,0.314226
KNNBaseline,3.255119,0.023144,0.255004,0.307208
BaselineOnly,15.158602,0.002932,0.015331,0.065969
CoClustering,39.480485,0.043937,0.005599,0.025329
KNNWithMeans,40.769536,0.024938,0.233028,0.024528
KNNWithZScore,58.791083,0.029751,0.235749,0.017009
NMF,111.283343,0.045479,0.005028,0.008986
NormalPredictor,126.754524,0.002486,0.004109,0.007889


In [10]:
train_formula, test_formula = train_test_split(data_formula, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)
train_complexity, test_complexity = train_test_split(data_complexity, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)
train_complexity12, test_complexity12 = train_test_split(data_complexity12, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)

In [11]:
from surprise import BaselineOnly, KNNBasic, KNNWithMeans, SVD, NMF, NormalPredictor, KNNBaseline, KNNWithZScore, SVDpp, CoClustering, SlopeOne

model1 = create_and_evaluate_model(train_formula, test_formula, CoClustering())
model2 = create_and_evaluate_model(train_complexity, test_complexity, NormalPredictor())
model3 = create_and_evaluate_model(train_complexity12, test_complexity12, SVD())

RMSE: 0.2000
MSE: 0.0400
MAE:  0.1951

RMSE: 14.6731
MSE: 215.2994
MAE:  11.0461

RMSE: 980.6927
MSE: 961758.1842
MAE:  975.6870



In [12]:
# 0 | 1305
estudiante_1 = 1305
print(f'Ejercicios realizados por el usuario [ {estudiante_1} ]')
df_catalogo[df_catalogo['oid'].isin(df_ratings[df_ratings['rut'] == estudiante_1]['oid'].values)].sort_values(by=["complexity"], ascending=True)

Ejercicios realizados por el usuario [ 1305 ]


,oid,nombre,hito,skill,knowledge,complexity,complexity12,formula,complexity_norm,complexity12_norm
25,89f44e7f5842479fb283e43c52ce067b,Ordenar tres nÃºmeros,1,0,1,1,257,0.190601,0.000000,0.000000
23,80c61dae74fa4915bf272ab17dfa62ff,Conversor de Decimal a Binario,1,0,2,2,258,0.171904,0.005051,0.001035


In [13]:
recomendaciones_estudiante_1 = get_top_n_recommendations(model1, df_ratings, feature_users='rut', feature_items='oid', user_id=estudiante_1)
print(f'Estudiante: {estudiante_1}')
df_catalogo[df_catalogo['oid'].isin(recomendaciones_estudiante_1)]

Estudiante: 1305


,oid,nombre,hito,skill,knowledge,complexity,complexity12,formula,complexity_norm,complexity12_norm
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,1,0,1,1,257,0.191384,0.000000,0.000000
6,29f15ef8dc32426f945f64e28c910a57,Suma de los divisores de un nÃºmero,2,1,3,19,531,0.224705,0.090909,0.283644
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,1,0,1,1,257,0.050914,0.000000,0.000000
8,37e4f4a1e8174e9496d21b00d67fc8f1,Distancia Levenshtein,2,3,7,55,567,0.262626,0.272727,0.320911
9,3d7d9e64ac0846caadc08dccf2538f55,Validar Secuencias de ADN,2,2,7,39,551,0.251442,0.191919,0.304348
11,4d51d13b9a2848a2803f7d1143fde6d4,Adivina la palabra,2,2,7,39,551,0.250398,0.191919,0.304348
12,4d5ed23727c04a8790f2e4ab3f09767e,FunciÃ³n buscarTodas,2,2,7,39,551,0.258492,0.191919,0.304348
21,7da6cedd04c44a15b5e421440253acff,El antipoema,2,2,7,39,551,0.250137,0.191919,0.304348
31,abc052e584734d0f8121d5e5ca659f82,Carro de Compras,2,2,7,39,551,0.241521,0.191919,0.304348
48,e72db1cb2e9f400990cfa3b464d0391a,Sopa de Letras,2,5,7,87,599,0.291521,0.434343,0.354037


In [14]:
recomendaciones_estudiante_1 = get_top_n_recommendations(model2, df_ratings, feature_users='rut', feature_items='oid', user_id=estudiante_1)
print(f'Estudiante: {estudiante_1}')
df_catalogo[df_catalogo['oid'].isin(recomendaciones_estudiante_1)]

Estudiante: 1305


,oid,nombre,hito,skill,knowledge,complexity,complexity12,formula,complexity_norm,complexity12_norm
1,16f619db31204ded9418136c4587ddd8,Calculadora GeomÃ©trica,2,1,1,17,529,0.188468,0.080808,0.281573
3,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,1,0,3,3,259,0.177229,0.010101,0.002070
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,1,0,1,1,257,0.050914,0.000000,0.000000
8,37e4f4a1e8174e9496d21b00d67fc8f1,Distancia Levenshtein,2,3,7,55,567,0.262626,0.272727,0.320911
9,3d7d9e64ac0846caadc08dccf2538f55,Validar Secuencias de ADN,2,2,7,39,551,0.251442,0.191919,0.304348
10,46850a246d48484b8f104f8aab5679b6,Descomponer un nÃºmero,1,0,3,3,259,0.192111,0.010101,0.002070
12,4d5ed23727c04a8790f2e4ab3f09767e,FunciÃ³n buscarTodas,2,2,7,39,551,0.258492,0.191919,0.304348
32,ac7382763e484d37908da54c076f7577,Cajero AutomÃ¡tico Nivel 1,1,0,3,3,259,0.138847,0.010101,0.002070
35,baf2f8e0167a4e089d2cec16582c9ae9,Factores Primos,1,1,3,19,275,0.142981,0.090909,0.018634
48,e72db1cb2e9f400990cfa3b464d0391a,Sopa de Letras,2,5,7,87,599,0.291521,0.434343,0.354037


In [15]:
recomendaciones_estudiante_1 = get_top_n_recommendations(model3, df_ratings, feature_users='rut', feature_items='oid', user_id=estudiante_1)
print(f'Estudiante: {estudiante_1}')
df_catalogo[df_catalogo['oid'].isin(recomendaciones_estudiante_1)]

Estudiante: 1305


,oid,nombre,hito,skill,knowledge,complexity,complexity12,formula,complexity_norm,complexity12_norm
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,1,0,1,1,257,0.191384,0.000000,0.000000
6,29f15ef8dc32426f945f64e28c910a57,Suma de los divisores de un nÃºmero,2,1,3,19,531,0.224705,0.090909,0.283644
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,1,0,1,1,257,0.050914,0.000000,0.000000
8,37e4f4a1e8174e9496d21b00d67fc8f1,Distancia Levenshtein,2,3,7,55,567,0.262626,0.272727,0.320911
9,3d7d9e64ac0846caadc08dccf2538f55,Validar Secuencias de ADN,2,2,7,39,551,0.251442,0.191919,0.304348
11,4d51d13b9a2848a2803f7d1143fde6d4,Adivina la palabra,2,2,7,39,551,0.250398,0.191919,0.304348
12,4d5ed23727c04a8790f2e4ab3f09767e,FunciÃ³n buscarTodas,2,2,7,39,551,0.258492,0.191919,0.304348
21,7da6cedd04c44a15b5e421440253acff,El antipoema,2,2,7,39,551,0.250137,0.191919,0.304348
31,abc052e584734d0f8121d5e5ca659f82,Carro de Compras,2,2,7,39,551,0.241521,0.191919,0.304348
48,e72db1cb2e9f400990cfa3b464d0391a,Sopa de Letras,2,5,7,87,599,0.291521,0.434343,0.354037
